In [2]:
from kv.api import KV
from kv.rest import ClientKV

In [9]:
kv = KV.of('http://localhost:8000/blobs')

In [6]:
for i in range(10):
  (await kv.insert(str(i), b'world')).unsafe()

In [7]:
await kv.clear()

Right(value=None, tag='right')